In [ ]:
import spacy
import json
import re
import pandas as pd
import nltk
# import wikipedia as wk
import time
# from pywikipedia import wikipedia as pw
import wikipediaapi as wk
# https://pypi.org/project/Wikipedia-API/


def load_json():

    with open("./gg2013.json") as f:
        json_obj = json.load(f)

    return list(set([ content["text"] for content in json_obj ]))

def cleansing_1(str_list):

    http_pattern = re.compile("(\w+:\/\/\S+)")
    hash_pattern = re.compile("(#[A-Za-z0-9_]+)")
    amp_pattern = re.compile("&([0-9a-zA-Z]+)")
    tag_pattern = re.compile("(@[A-Za-z0-9_]+)")
    rt_pattern = re.compile("RT @[a-zA-Z0-9_]+: ")
    rt_pattern_2 = re.compile("RT")

    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)

    return_txt_list = []

    for i in str_list:

        v1 = re.sub(http_pattern, "", i)
        # v2 = re.sub(hash_pattern, "", v1)
        v3 = re.sub(amp_pattern, "", v1)
        v4 = re.sub(rt_pattern, "", v3)
        v5 = re.sub(tag_pattern, "", v4)
        v6 = re.sub(emoji_pattern, "", v5)
        v7 = re.sub(rt_pattern_2, "", v6)
        v8 = re.sub(r"[^a-zA-Z_ ]", "", v7)

        if len(v8) > 2:

            return_txt_list.append(v8.strip())

    return return_txt_list

def cleansing_2(str_list):

    keywords_1 = ["nominated", "nominee", "nominate", "nomination", "nominees"]
    # keywords_2 = ["best", "win", "wins"]
    # return [txt for txt in str_list if any(keyword in txt.lower() for keyword in keywords_1) and any(keyword in txt.lower() for keyword in keywords_2)]
    return [txt for txt in str_list if any(keyword in txt.lower() for keyword in keywords_1) ]
    # return [ txt for txt in str_list if any(keyword in txt.lower() for keyword in keywords_1) ]

def nltk_download():

    nltk.download('punkt')
    nltk.download('averaged_perceptron_tagger')
    nltk.download('stopwords')
    nltk.download('wordnet')  # Lemmatization
    nltk.download('omw-1.4')  # Lemmatization
    # nltk.download('maxent_ne_chunker')
    # nltk.download('words')

    print("NLTK Download END")

def nltk_filter(str_list):

    wlem = nltk.WordNetLemmatizer()
    nltk_list = []

    with open("basic_stopword.txt", "r", encoding="utf-8") as f:
        stopword = f.read()
    stopword_list = stopword.split()

    '''
    pattern = re.compile(r"[^\w'.,:;]|_")

    for index, value in enumerate(str_list):
        str_list[index] = " ".join(re.sub(pattern, " ", value).strip().split())
    '''

    for index, value in enumerate(str_list):

        # print(index)

        topic_list = []

        for pos in nltk.pos_tag(nltk.word_tokenize(value)):
             # if ( pos[0]. startswith("best") ):
             #   print(pos)
             # if (pos[1].startswith("JJ") or pos[1].startswith("NN") or pos[1].startswith("RB") or pos[1].startswith(
             #        "VB") ) \
             #        and ("'" not in pos[0]):
                ori_keyword = pos[0].strip()
                pre_keyword = pos[0].lower().strip()  # pos_tagging -> keyword ( Wayne )

                if pre_keyword in stopword_list:
                    continue
                if len(pre_keyword) < 3:
                    continue

                keyword = wlem.lemmatize(pre_keyword)  # keyword -> lemmatize -> keyword ( Wa )


                if len(pre_keyword) < 3:
                    continue
                if keyword not in stopword_list:
                    # topic_list.append(keyword)
                    if pos[1].startswith("NN"):
                        topic_list.append(ori_keyword)
                    else :
                        topic_list.append(ori_keyword.lower())

        nltk_list.append(" ".join(topic_list))

    return nltk_list

def cleansing_3(str_list):


    nlp = spacy.load("en_core_web_sm")

    person_list = []

    for txt in str_list:
        # print(txt)
        for entity in nlp(txt).ents:
            # if entity.label_ == "PERSON" or entity.label_ == "ORG" or entity.label_ == "PRODUCT" or entity.label_ == "WORD_OF_ART":
            if entity.label_ == "PERSON":
                ent_str_lower = str(entity).lower()
                if "win" not in ent_str_lower and "best" not in ent_str_lower:
                    ent_str = re.sub(r"[^a-zA-Z0-9_ ]", "", str(entity))
                    person_list.append([txt, ent_str.strip(), entity.label_])


    return person_list

def load_str_4():

    df = pd.read_csv("./sample_str_4.csv")
    ans = pd.read_csv("./answers.csv")

    #############################
    name_list = []

    for i in ans.index:
        named = ans.loc[i, "nominees"][1:-1].split(", ")
        for i in named:
            if len(i.split(" ")) < 3: # Person name limit
                name_list.append(i)

    print(name_list)
    print(len(name_list))
    print("------ award name list (127) --------")
    #############################

    unique_dict = {}

    for i in df.index:
        dict_key = df.loc[i, "Entity"].lower()

        if len(dict_key.split(" ")) < 3: # Person Name Limit

            if dict_key not in unique_dict.keys():
                unique_dict[dict_key] = 1
            elif dict_key in unique_dict.keys():
                unique_dict[dict_key] += 1
    print("len(unique_dict) : ", len(unique_dict))
    print("len(unique_dict).intersection : ", len(set(list(unique_dict)).intersection(set(name_list))))

    unique_dict = {key: value for key, value in unique_dict.items() if value > 1}

    # unique_dict = {key: value for key, value in unique_dict.items() if value > 1}

    wiki_dict = {}
    wkk = wk.Wikipedia('en')

    for k, v in unique_dict.items():
        # print(k, v)
        k_split = str(k).title().split(" ")
        if len(k_split) != 1:
            k_wiki = "_".join(str(k).title().split(" ")).strip()
        else :
            continue
        # print(k_wiki.strip())
        # k_wiki = str(k).title()
        # print(k_wiki)
        # if wk.suggest(k_wiki) != None:
        #    wiki_dict[k] = v
        # try:
        #     print(wk.page(k_wiki))
        # except:
        #     print("그런 위키는 없어요")
        # else:
        #     wiki_dict[k] = v

        if wkk.page(k_wiki).exists() == True:
            # print(k_wiki)
            wiki_dict[k] = v

    print("len(wiki_dict) : ", len(wiki_dict))
    print("len(wiki_dict).intersection : ", len(set(list(wiki_dict)).intersection(set(name_list))))
    # Success
    # unique_dict = {key:value for key, value in unique_dict.items() if value > 1}

    print("len(unique_dict) : ", len(unique_dict))
    print("len(unique_dict).intersection : ", len(set(list(unique_dict)).intersection(set(name_list))))

    print("--------------------------")

    '''
    unique_dict_2 = {}

    for k_1, v_1 in unique_dict.items():
        alive = k_1
        num = v_1
        for k_2, v_2 in unique_dict.items():
            if k_1 in k_2 or k_2 in k_1:
                if ( k_1 != k_2 ):
                    if v_1 > v_2:
                        alive = k_1
                        num = v_1
                    elif v_2 > v_1:
                        alive = k_2
                        num= v_2

        if alive not in unique_dict_2.keys():
            # unique_dict_2[alive] = num
            exist = False
            for k, v in unique_dict_2.items():

                if ( str(alive) in str(k) ) or ( str(k) in str(alive) ):

                    exist = True

                    if v != 0:
                        if num > v:
                            exist = False
                            # unique_dict_2[alive] = num
                            unique_dict_2[k] = 0

                        elif v > num :
                            continue
            if exist == False:
                unique_dict_2[alive] = num

    # unique_dict_2 = {key:value for key, value in unique_dict_2.items() if value != 0}

    print("unique_dict_2")
    print("len(unique_dict_2) : ", len(unique_dict_2))
    print("len(set(list(unique_dict_2)).intersection : ", len(set(list(unique_dict_2)).intersection(set(name_list))))
    
    
    filter_dict = {key.lower(): value for key, value in unique_dict_2.items() if value != 0 }

    print("filter_dict")
    print("len(filter_dict) : ", len(filter_dict))
    print("len(set(list(filter_dict)).intersection : ", len(set(list(filter_dict)).intersection(set(name_list))))
    '''

    ################################



    # 1 - 63 all - 72 / 127
    # print(len(set(list(filter_dict)).intersection(set(name_list))))
    # print(len(set(list(unique_dict)).intersection(set(name_list))))
    # print(len(set(list(unique_dict_2)).intersection(set(name_list))))
    print("END")
    # if wk.suggest("sdfsdfsdfsdfsdf") == None :
    #    print("hello")
    wiki_dict
def main():


    str_list = load_json()
    str_list_2 = cleansing_1(str_list)

    str_2 = pd.DataFrame(str_list_2, columns=["Full_Text"])
    str_2 = str_2.dropna(axis=0)
    str_2.to_csv("./sample_str_2.csv", index=False)

    str_list_3 = cleansing_2(str_list_2)

    str_3 = pd.DataFrame(str_list_3, columns=["Full_Text"])
    str_3 = str_3.dropna(axis=0)
    str_3.to_csv("./sample_str_3.csv", index=False)

    nltk_download()
    nltk_list = nltk_filter(str_list_3)

    nltk_df = pd.DataFrame(nltk_list, columns=["Full_Text"])
    nltk_df = nltk_df.dropna(axis=0)
    nltk_df.to_csv("./nltk_list.csv", index=False)

    str_list_4 = cleansing_3(nltk_list)

    str_4 = pd.DataFrame(str_list_4, columns=["Full_Text", "Entity", "Entity_Label"])
    str_4 = str_4.dropna(axis=0)
    str_4 = str_4.astype("str")
    str_4 = str_4.sort_values(by=["Full_Text"], ascending=True)
    str_4.to_csv("./sample_str_4.csv", index=False)



    load_str_4()

if __name__ == "__main__":
    main()